# 3D Point Cloud Simulation of 2D KITTI LiDAR Dataset using Open3D

In [3]:
# Import library functions
import os
import time
import requests
import math

import matplotlib.pyplot as plt
import numpy as np
import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [4]:
# Function definition to download the dataset
def download_dataset(url, local_filename):

    # Update Dropbox link to force download
    if "www.dropbox.com" in url and "?dl=0" in url:
        url = url.replace("?dl=0", "?dl=1")
    
    # Send a GET request to the URL
    response = requests.get(url)
    
    # Check if the request was successful
    if response.status_code == 200:
        # Write the content of the response to a file
        with open(local_filename, 'wb') as f:
            f.write(response.content)
        print(f"File downloaded and saved as {local_filename}")
    else:
        print(f"Failed to download file. Status code: {response.status_code}")

In [ ]:
# Download 2D KITTI Depth Frames Dataset 
# download_dataset('https://www.dropbox.com/scl/fi/wfg0ta7kx57be15hw40wl/archive.zip?rlkey=fei6eqeucmbcbkw478dfsy7qg&dl=1', 'archive.zip')

Read 2D Depth Image

In [ ]:
# Read the 2D Depth Image
def load_depth_image(file_path):
    # Load the depth image
    depth_image = plt.imread(file_path)

    depth_image_scaling_factor = 250.0
    # Assuming the depth image is normalized, we may need to scale it to the actual distance values
    # This scaling factor is dataset-specific; you'll need to adjust it based on the KITTI dataset documentation
    depth_image *= depth_image_scaling_factor
    
    return depth_image

Converting 2D Depth Frames into 3D Point Cloud

In [ ]:
def depth_image_to_point_cloud(depth_image, h_fov=(-90, 90), v_fov=(-24.9, 2.0), d_range=(0,100)):
    # Adjusting angles for broadcasting
    h_angles = np.deg2rad(np.linspace(h_fov[0], h_fov[1], depth_image.shape[1]))
    v_angles = np.deg2rad(np.linspace(v_fov[0], v_fov[1], depth_image.shape[0]))

    # Reshaping angles for broadcasting
    h_angles = h_angles[np.newaxis, :]  # Shape becomes (1, 1440)
    v_angles = v_angles[:, np.newaxis]  # Shape becomes (64, 1)

    # Calculate x, y, and z
    x = depth_image * np.sin(h_angles) * np.cos(v_angles)
    y = depth_image * np.cos(h_angles) * np.cos(v_angles)
    z = depth_image * np.sin(v_angles)

    # Filter out points beyond the distance range
    valid_indices = (depth_image >= d_range[0]) & (depth_image <= d_range[1])
    
    # Apply the mask to each coordinate array
    x = x[valid_indices]
    y = y[valid_indices]
    z = z[valid_indices]

    # Stack to get the point cloud
    point_cloud = np.stack((x, y, z), axis=-1)

    return point_cloud

Load and Process Frames

In [ ]:
def load_and_process_frames(directory):
    point_clouds = []
    for filename in sorted(os.listdir(directory)):
        if filename.endswith('.png'):  # Check for PNG images
            file_path = os.path.join(directory, filename)
            depth_image = load_depth_image(file_path)
            point_cloud = depth_image_to_point_cloud(depth_image)
            point_clouds.append(point_cloud)
    return point_clouds

Simulate Point Cloud Representation

In [ ]:
def animate_point_clouds(point_clouds):
    vis = o3d.visualization.Visualizer()
    vis.create_window()

    # Set background color to black
    vis.get_render_option().background_color = np.array([0, 0, 0])

    # Initialize point cloud geometry
    point_cloud = o3d.geometry.PointCloud()
    point_cloud.points = o3d.utility.Vector3dVector(point_clouds[0])
    vis.add_geometry(point_cloud)

    frame_index = 0
    last_update_time = time.time()
    update_interval = 0.25  # Time in seconds between frame updates

    while True:
        current_time = time.time()
        if current_time - last_update_time > update_interval:
            # Update point cloud with new data
            point_cloud.points = o3d.utility.Vector3dVector(point_clouds[frame_index])
            vis.update_geometry(point_cloud)

            # Move to the next frame
            frame_index = (frame_index + 1) % len(point_clouds)
            last_update_time = current_time

        vis.poll_events()
        vis.update_renderer()

        if not vis.poll_events():
            break

    vis.destroy_window()

In [ ]:
# Directory containing the depth image files
directory = 'archive/2011_09_30_drive_0028_sync/2011_09_30_drive_0028_sync/2011_09_30/2011_09_30_drive_0028_sync/velodyne_points/depth_images'

# Load and process the frames
point_clouds = load_and_process_frames(directory)

# Simulate the point clouds
animate_point_clouds(point_clouds)